# Sensor Data
Transfer from all 12 Racks over 6 years worth of csv into one pandas dataframe.
pd.concat needs to happen outside the loop, or the time will exponentially slow down
due to the way pandas copies dataframes.

In [ ]:
import pandas as pd
import os
import re
from src.data.dataframe import createDataFrame, formatDataFrame
from sqlalchemy import create_engine


directory = "L:/Cavefish/Facility/Life Support Systems/PLC Data Logs/Individual Rack Data"

# dataframe creation
sensor_df = pd.DataFrame(columns={'rack_num', 'Date_Time',
                         'Level', 'Temperature', 'pH', 'Conductivity', 'Flow', 'DO'})

# create a list to house all the csv dataframes created
super_df = []

# walk through all the subdirectories and files
for root, dirs, files in os.walk(directory):
    for filename in files:
        rack_number = re.search(r'CF(.*?)\.', filename)[1]
        try:
            if 'Sensor' in filename:
                print(os.path.join(root, filename))
                df_temp = createDataFrame()
                super_df.append(df_temp)
        except (TypeError) as error:
            print(error)
            continue

sensor_df = formatDataFrame(super_df)
sensor_df.drop(['DO'], axis=1, inplace=True)
sensor_df = sensor_df[['rack_num', 'Date_Time', 'pH',
                                       'Conductivity', 'Temperature', 'Flow', 'Level']]
sensor_df.drop_duplicates(
     subset=['Date_Time'], inplace=True)

%store sensor_df